In [1]:
import os
import random
import time
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

location = 'BJ'
Type = 'PM10'

predictors=[
#'longtitude',
# 'latitude',
 'PM2.5_t1',
 'PM10_t1',
 'O3_t1',
 'knn1_PM2.5_t1',
 'knn1_PM10_t1',
 'knn1_O3_t1',
 'knn2_PM2.5_t1',
 'knn2_PM10_t1',
 'knn2_O3_t1',
 'knn3_PM2.5_t1',
 'knn3_PM10_t1',
 'knn3_O3_t1',
 'knn4_PM2.5_t1',
 'knn4_PM10_t1',
 'knn4_O3_t1',
 'hour_sin',
 'hour_cos',
 'week_day_sin',
 'week_day_cos',  
]

#target
target = []
for i in range(6,37):
    target.append(Type+ '_t'+str(i))#'PM2.5_t7'

input_size = len(predictors)
output_size = len(target)
lr = 0.005
per = 0.1
batch_size = 64
time_step = 5

rnn_unit = 30
w1size = 40#fully conective network size 
w2size = 30
w3size = 30

f = open('/home/dedekinds/data_BJ_sample_LSTM.csv')
df = pd.read_csv(f)
df = df.dropna(axis=0,how='any')
#data = df.values
#df = df[:100]#测试

INPUT_DATA = df[predictors].values
OUTPUT_DATA = df[target].values

def get_train_data(batch_size,time_step):
    print('getting data')
    batch_index = []
    #data_train = train_data_split
    #normalized_train_data = (INPUT_DATA[:,:-4]-np.mean(INPUT_DATA[:,:-4],axis=0))/(0.0001+np.std(INPUT_DATA[:,:-4],axis=0))#+0.00001   /0
    #normalized_train_data = np.column_stack((normalized_train_data,INPUT_DATA[:,-4:]))
    train_x = []
    train_y = []
    for i in range(len(INPUT_DATA)-time_step):
        if i % batch_size==0:
            batch_index.append(i)
        x = INPUT_DATA[i:i+time_step ,:input_size]
        x[:,:-4] = (x[:,:-4]-np.min(x[:,:-4],axis=0))/(0.0001+np.max(x[:,:-4],axis=0)-np.min(x[:,:-4],axis=0))
        #y = OUTPUT_DATA[i:i+time_step ,:]#,np.newaxis
        y = OUTPUT_DATA[i:i+time_step ,:]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    batch_index.append(len(INPUT_DATA)-time_step)
    return batch_index,train_x,train_y



weights={
         'in':tf.Variable(tf.random_normal([input_size,rnn_unit])),
         'w1':tf.Variable(tf.random_normal([rnn_unit,w1size])),
         'w2':tf.Variable(tf.random_normal([w1size,w2size])),
#         'w3':tf.Variable(tf.random_normal([w2size,w3size])), 
         'out':tf.Variable(tf.random_normal([w2size,output_size]))
        }
biases={
        'in':tf.Variable(tf.constant(0.1,shape=[rnn_unit,])),
        'w1':tf.Variable(tf.constant(0.1,shape=[w1size,])),
        'w2':tf.Variable(tf.constant(0.1,shape=[w2size,])),
#        'w3':tf.Variable(tf.constant(0.1,shape=[w3size,])), 
        'out':tf.Variable(tf.constant(0.1,shape=[output_size,]))
       }


def lstm(X,keep_prob):     
    batch_size=tf.shape(X)[0]
    time_step=tf.shape(X)[1]
    w_in=weights['in']
    b_in=biases['in']  
    input=tf.reshape(X,[-1,input_size])  #需要将tensor转成2维进行计算，计算后的结果作为隐藏层的输入
    input_rnn=tf.matmul(input,w_in)+b_in
    input_rnn=tf.reshape(input_rnn,[-1,time_step,rnn_unit])  #将tensor转成3维，作为lstm cell的输入
    cell=tf.nn.rnn_cell.BasicLSTMCell(rnn_unit)
    init_state=cell.zero_state(batch_size,dtype=tf.float32)
    output_rnn,final_states=tf.nn.dynamic_rnn(cell, input_rnn,initial_state=init_state, dtype=tf.float32)  #output_rnn是记录lstm每个输出节点的结果，final_states是最后一个cell的结果
    output=tf.reshape(output_rnn,[-1,rnn_unit]) #作为输出层的输入

    w1 = weights['w1']
    b1 = biases['w1']
    w1_plus_b = tf.matmul(output, w1) + b1
    w1_plus_b = tf.nn.dropout(w1_plus_b, keep_prob)
    w1_output = tf.nn.relu( w1_plus_b, )
    output1=tf.reshape(w1_output,[-1,w1size]) 
    
    w2 = weights['w2']
    b2 = biases['w2']
    w2_plus_b = tf.matmul(output1, w2) + b2
    w2_plus_b = tf.nn.dropout(w2_plus_b, keep_prob)
    w2_output = tf.nn.relu( w2_plus_b, )
    output2=tf.reshape(w2_output,[-1,w2size]) 
    
#    w3 = weights['w3']
#    b3 = biases['w3']
#    w3_plus_b = tf.matmul(output2, w3) + b3
#    w3_plus_b = tf.nn.dropout(w3_plus_b, keep_prob)
#    w3_output = tf.nn.relu( w3_plus_b, )
#    output3=tf.reshape(w3_output,[-1,w3size]) 

    w_out=weights['out']
    b_out=biases['out']
    pred=tf.matmul(output2,w_out)+b_out
    return pred,final_states


def train_lstm(batch_size,time_step):
    print('start training')
    keep_prob = tf.placeholder(tf.float32)
    X=tf.placeholder(tf.float32, shape=[None,time_step,input_size])
    Y=tf.placeholder(tf.float32, shape=[None,time_step,output_size])
    batch_index,train_x,train_y=get_train_data(batch_size,time_step)
    pred,_=lstm(X,keep_prob)
    #损失函数
    loss=tf.reduce_mean(tf.square(tf.reshape(pred,[-1])-tf.reshape(Y, [-1])))
    train_op=tf.train.AdamOptimizer(lr).minimize(loss)
#    saver=tf.train.Saver(tf.global_variables(),max_to_keep=15)
#    module_file = tf.train.latest_checkpoint()    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        saver = tf.train.Saver()
        tf.add_to_collection('X', X)
        tf.add_to_collection('keep_prob', keep_prob)
        tf.add_to_collection('pred', pred)
                
        #重复训练10000次
        for i in range(30):
            for step in range(len(batch_index)-1):
                _,loss_=sess.run([train_op,loss],feed_dict={X:train_x[batch_index[step]:batch_index[step+1]],Y:train_y[batch_index[step]:batch_index[step+1]],keep_prob:0.9})
                #if i % 100 == 0:
            print(i,loss_)
#            if i % 200==0:
#                print("保存模型：",saver.save(sess,'stock2.model',global_step=i))
        model_dir = "/home/dedekinds/LSTM拼接"
        model_name = Type+'_lstm_ann_'+location
        if not os.path.exists(model_dir):
            os.mkdir(model_dir)
        # 保存模型
        saver.save(sess, os.path.join(model_dir, model_name))
        print("保存模型成功！")

        
start_time = time.time()
train_lstm(batch_size,time_step)
print('[{}]: model training time'.format(time.time() - start_time))

/home/dedekinds/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


start training
getting data
0 2019.1301
1 2253.5
2 2498.0818
3 2158.3005
4 2249.334
5 2132.9468
6 2116.582
7 2169.0588
8 2379.144
9 2254.2246
10 2313.7305
11 2276.2102
12 2181.944
13 2503.5125
14 2252.8586
15 2347.368
16 2301.4438
17 2215.016
18 2368.248
19 2303.664
20 2434.6897
21 2424.5608
22 2295.5613
23 2510.0193
24 2843.433
25 2445.8147
26 2381.8174
27 2614.9028
28 2689.2153
29 2341.746
保存模型成功！
[863.1624479293823]: model training time
